In [2]:
!pip install open_clip_torch

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 73.8 MB/s eta 0:00:00


In [1]:
from llava import LlavaLlamaForCausalLM
from transformers import CLIPVisionModel, CLIPImageProcessor, StoppingCriteria
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig
import torch
import os
import pickle
import math

def patch_config(config):
    patch_dict = {
        "use_mm_proj": True,
        "mm_vision_tower": "openai/clip-vit-large-patch14",
        "mm_hidden_size": 1024
    }

    cfg = AutoConfig.from_pretrained(config)
    if not hasattr(cfg, "mm_vision_tower"):
        print(f'`mm_vision_tower` not found in `{config}`, applying patch and save to disk.')
        for k, v in patch_dict.items():
            setattr(cfg, k, v)
        cfg.save_pretrained(config)
        
model_name = '/data/user-data/sa25729/.cache/huggingface/hub/models--microsoft--llava-med-v1.5-mistral-7b/snapshots/f2f72301dc934e74948b5802c87dbc83d100e6bd'
# patch_config(model_name)
# model_name = 'liuhaotian/LLaVA-7b-delta-v0'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = LlavaLlamaForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, use_cache=True).cuda()
# model = AutoModelForCausalLM.from_pretrained("microsoft/llava-med-7b-delta").cuda()

image_processor = CLIPImageProcessor.from_pretrained(model.config.mm_vision_tower, torch_dtype=torch.float16)

# Load model directly
from transformers import AutoProcessor, AutoModelForCausalLM

processor = AutoProcessor.from_pretrained("microsoft/llava-med-7b-delta")


vision_tower = model.model.vision_tower[0]
vision_tower.to(device='cuda', dtype=torch.float16)
DEFAULT_IMAGE_TOKEN = "<image>"
DEFAULT_IMAGE_PATCH_TOKEN = "<im_patch>"
DEFAULT_IM_START_TOKEN = "<im_start>"
DEFAULT_IM_END_TOKEN = "<im_end>"


mm_use_im_start_end = getattr(model.config, "mm_use_im_start_end", False)
# print('mm_use_im_start_end', mm_use_im_start_end) True
tokenizer.add_tokens([DEFAULT_IMAGE_PATCH_TOKEN], special_tokens=True)
if mm_use_im_start_end:
    tokenizer.add_tokens([DEFAULT_IM_START_TOKEN, DEFAULT_IM_END_TOKEN], special_tokens=True)



2024-09-27 12:21:53.149942: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-27 12:21:53.993870: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
You are using a model of type llava_mistral to instantiate a model of type llava. This is not supported for all configurations of models and can yield errors.
/data/user-data/sa25729/.local/lib/python3.8/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of LlavaLlamaForCausalLM were not initialized from the model checkpoint at /data/user-data/sa25729/.cache/huggingface/hub/models--microsoft--llava-med-v1.5-mistral-7b/snapshots/f2f72301dc934e74948b5802c87dbc83d100e6bd and are newly initialized: ['model.mm_projector.bias', 'model.mm_projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


576


In [5]:
# import pdb; pdb.set_trace()
vision_config = vision_tower.config
vision_config.im_patch_token = tokenizer.convert_tokens_to_ids([DEFAULT_IMAGE_PATCH_TOKEN])[0]
vision_config.use_im_start_end = mm_use_im_start_end
if mm_use_im_start_end:
    vision_config.im_start_token, vision_config.im_end_token = tokenizer.convert_tokens_to_ids([DEFAULT_IM_START_TOKEN, DEFAULT_IM_END_TOKEN])
image_token_len = (vision_config.image_size // vision_config.patch_size) ** 2

print(vision_config.image_size, vision_config.patch_size, image_token_len)

336 14 576


In [2]:
question_file = '/data/user-data/sa25729/LLaVA-Med/data/pvqa/qas/test/test_qa.pkl'
with open(os.path.expanduser(question_file), "rb") as f:
    questions_all = pickle.load(f)
print(len(questions_all), questions_all[0])

def split_list(lst, n):
    """Split a list into n (roughly) equal-sized chunks"""
    chunk_size = math.ceil(len(lst) / n)  # integer division
    return [lst[i:i+chunk_size] for i in range(0, len(lst), chunk_size)]

def get_chunk(lst, n, k):
    chunks = split_list(lst, n)
    return chunks[k]

num_chunks  = 1
chunk_idx = 0
questions = get_chunk(questions_all, num_chunks, chunk_idx)

print(len(questions), questions[0])

#save answer file
answers_file_path = '/data/user-data/sa25729/LLaVA-Med/output_data/pvqa/answer-file-pvqa-temp.json'
answers_file = os.path.expanduser(answers_file_path)
os.makedirs(os.path.dirname(answers_file), exist_ok=True)
os.makedirs(os.path.join(os.path.dirname(answers_file), "images"), exist_ok=True)
ans_file = open(answers_file, "w")
save_image_folder = os.path.join(os.path.dirname(os.path.expanduser(answers_file)), "images")

6761 {'image': 'test_0167', 'question': 'What are positively charged,  thus allowing the compaction of the negatively charged DNA?', 'answer': 'the histone subunits'}
6761 {'image': 'test_0167', 'question': 'What are positively charged,  thus allowing the compaction of the negatively charged DNA?', 'answer': 'the histone subunits'}


In [3]:
from PIL import Image
from llava.conversation import conv_templates
from transformers import CLIPVisionModel, CLIPImageProcessor, StoppingCriteria
import shortuuid
import json


class KeywordsStoppingCriteria(StoppingCriteria):
    def __init__(self, keywords, tokenizer, input_ids):
        self.keywords = keywords
        self.tokenizer = tokenizer
        self.start_len = None
        self.input_ids = input_ids

    def __call__(self, output_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        if self.start_len is None:
            self.start_len = self.input_ids.shape[1]
        else:
            outputs = self.tokenizer.batch_decode(output_ids[:, self.start_len:], skip_special_tokens=True)[0]
            for keyword in self.keywords:
                if keyword in outputs:
                    return True
        return False

image_folder = '/data/user-data/sa25729/LLaVA-Med/data/pvqa/images/test'
conv_mode = 'simple'

question = questions[0]["question"].split('\n')[0]
gt_ans = questions[0]["answer"] # ['value'] 
qs = question
qs = qs.replace('<image>', '').strip()
cur_prompt = qs

image_file = questions[0]["image"]
image = Image.open(os.path.join(image_folder, image_file + '.jpg'))
image_tensor = image_processor.preprocess(image, return_tensors='pt')['pixel_values'][0]
images = image_tensor.unsqueeze(0).half().cuda()
qs = qs + '\n' + DEFAULT_IMAGE_PATCH_TOKEN * image_token_len
cur_prompt = cur_prompt + '\n' + '<image>'

conv = conv_templates[conv_mode].copy()
conv.append_message(conv.roles[0], qs)
prompt = conv.get_prompt()
inputs = tokenizer([prompt])

input_ids = torch.as_tensor(inputs.input_ids).cuda()

keywords = ['###']
stopping_criteria = KeywordsStoppingCriteria(keywords, tokenizer, input_ids)

with torch.inference_mode():
    output_ids = model.generate(
        input_ids,
        images=images,
        do_sample=True,
        temperature=0.7,
        max_new_tokens=1024,
        stopping_criteria=[stopping_criteria])
    
input_token_len = input_ids.shape[1]
n_diff_input_output = (input_ids != output_ids[:, :input_token_len]).sum().item()
if n_diff_input_output > 0:
    print(f'[Warning] Sample {i}: {n_diff_input_output} output_ids are not the same as the input_ids')

outputs = tokenizer.batch_decode(output_ids[:, input_token_len:], skip_special_tokens=True)[0]

print('outputs', outputs)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
../aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [98,0,0], thread: [0,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [98,0,0], thread: [1,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [98,0,0], thread: [2,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [98,0,0], thread: [3,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [98,0,0], thread: [4,0,0] Assertion `srcIndex < srcSelectDimSize` failed

RuntimeError: cuDNN error: CUDNN_STATUS_NOT_INITIALIZED

In [4]:
!pwd

/data/user-data/sa25729/LLaVA-Med
